# PROJET ML — CLASSIFICATION DE VIDÉOS YOUTUBE *TRENDING*

**Dataset Kaggle**: https://www.kaggle.com/datasets/datasnaek/youtube-new

**Objectif**: Développer un classificateur pour prédire si une vidéo YouTube deviendra *trending*.

> Ce notebook est conçu pour être complété. Chaque section contient des consignes détaillées et des zones de code à compléter.


## 1. Importation des librairies

**Consigne 1.1 — Importez toutes les librairies nécessaires**
- `pandas`, `numpy` pour la manipulation de données  
- `matplotlib.pyplot`, `seaborn` pour la visualisation  
- `sklearn` pour le machine learning  
- `warnings` pour supprimer les avertissements

In [2]:
# VOTRE CODE ICI - Section 1.1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')
print("Kirill")

Kirill


## 2. Chargement et exploration des données

**Consigne 2.1 — Chargement des données**
- Chargez le fichier `USvideos.csv` avec pandas  
- Affichez les 5 premières lignes  
- Affichez les informations générales (`info()`, `shape`, `describe()`)

*Aide* :
- Utilisez `pd.read_csv()` avec `encoding='utf-8'`
- `.info()` donne les types de colonnes et valeurs non-nulles
- `.describe()` donne les statistiques descriptives

In [5]:
# VOTRE CODE ICI - Section 2.1

df = pd.read_csv('archive/USvideos.csv', encoding='utf-8')
print("Premières lignes: ", df.head())
display(df())
print("\nInformations générales:", df.info())
display(...())
print("Shape:",)
display(df.describe(include='all').transpose())

FileNotFoundError: [Errno 2] No such file or directory: 'archive/USvideos.csv'

**Consigne 2.2 — Analyse des valeurs manquantes**  
- Comptez les valeurs manquantes par colonne  
- Identifiez les colonnes avec le plus de valeurs manquantes  
- Affichez le pourcentage de valeurs manquantes

In [ ]:
# VOTRE CODE ICI - Section 2.2
# missing = df...
# pct_missing = (missing / len(df)) * 100
# print("Valeurs manquantes (nombre):")
# display(....))
# print("\nValeurs manquantes (pourcentage):")
# display(...))

**Consigne 2.3 — Exploration des catégories**  
- Chargez le fichier JSON des catégories (`US_category_id.json`)  
- Fusionnez avec le DataFrame principal  
- Affichez la distribution des catégories

*Aide* : La structure JSON est du type:  
`{"items": [{"id": "1", "snippet": {"title": "Film & Animation"}}, ...]}`

In [ ]:
# VOTRE CODE ICI - Section 2.3
# import pandas as pd
# import json
# categories_raw = read-json...
# items = categories_raw['items'].tolist()
# category_dict = {int(item['id']): item['snippet']['title'] for item in items}
# cat_df = pd.DataFrame(list(category_dict.items()), columns=['category_id', 'category_title'])
# df_merged = df.merge(cat_df, how='left', left_on='category_id', right_on='category_id')
# print("Distribution des catégories:")
# display(df_merged['category_title'].value_counts().to_frame('count'))

## 3. Nettoyage des données

**Consigne 3.1 — Nettoyage des données**
- Supprimez les doublons basés sur `video_id`  
- Gérez les valeurs manquantes dans `description` (remplacez par string vide)  
- Convertissez `publish_time` en datetime  
- Supprimez les lignes avec des valeurs aberrantes (ex: `views` négatives)

In [ ]:
# VOTRE CODE ICI - Section 3.1
# df_clean = df.drop_duplicates(subset='video_id').copy()
# df_clean['description'] = ....
# import pandas as pd
# df_clean['publish_time'] = ().... errors='coerce')
# df_clean = df_clean[(df_clean['views'] >= 0) & (df_clean['likes'] >= 0) & (df_clean['dislikes'] >= 0) & (df_clean['comment_count'] >= 0)]
# df_clean = df_clean.dropna(subset=['publish_time'])

## 4. Feature Engineering

**Consigne 4.1 — Variables d'engagement**
** définition : https://support.google.com/youtube/answer/2991785?hl=fr**

In [ ]:
# VOTRE CODE ICI - Section 4.1
# import numpy as np
# eps = 1e-9
# denom = df_clean['likes'] + df_clean['dislikes']
# df_clean['like_ratio'] = np.where(denom > 0, df_clean['likes'] / denom, 0)
# df_clean['engagement_rate'] = np.where(df_clean['views'] > 0,
#                                        (df_clean['likes'] + df_clean['dislikes'] + df_clean['comment_count']) / (df_clean['views'] + eps),
#                                        0)
# df_clean['comments_per_view'] = np.where(df_clean['views'] > 0, df_clean['comment_count'] / (df_clean['views'] + eps), 0)

**Consigne 4.2 — Variables temporelles**

In [ ]:
# VOTRE CODE ICI - Section 4.2
# df_clean['publish_hour'] = ....
# df_clean['publish_day_of_week'] = ...
# df_clean['publish_month'] = ...

**Consigne 4.3 — Variables textuelles (titre)**

In [ ]:
# VOTRE CODE ICI - Section 4.3
# df_clean['title_length'] = ...
# df_clean['title_word_count'] = ....
# df_clean['has_caps'] = ...
# df_clean['has_numbers'] = ...

## 5. Création de la variable cible

**Consigne 5.1 — Définition de `is_trending`**  
- Une vidéo est *trending* si elle a plus de vues que le 80e percentile de sa catégorie

In [ ]:
# VOTRE CODE ICI - Section 5.1
# df_clean['is_trending'] = (df_clean.groupby('category_id')['views']
#                                   .transform(lambda x: x > x.quantile(0.8))).astype(int)
# print("Distribution de la variable cible:")
# display(df_clean['is_trending'].value_counts().to_frame('count'))

## 6. Préparation des données pour l'entraînement

**Consignes 6.1 & 6.2 — Sélection des features & split train/test**

In [ ]:
# VOTRE CODE ICI - Section 6.1 et 6.2
# features = ['views', 'likes', 'dislikes', 'comment_count',
#             'like_ratio', 'engagement_rate', 'title_length', 'publish_hour', 'category_id']
# X = df_clean[features].copy()
# y = df_clean['is_trending'].copy()
#.......
# ......

# print(f"Taille train: {X_train.shape}")
# print(f"Taille test: {X_test.shape}")

**Consigne 6.3 — Normalisation des données (StandardScaler)**

In [ ]:
# VOTRE CODE ICI - Section 6.3
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# num_cols = ['views', 'likes', 'dislikes', 'comment_count', 'like_ratio', 'engagement_rate', 'title_length', 'publish_hour']
# # Fit uniquement sur train
# ....
# ...
# ....
# ....
# ....

## 7. Modèle 1 — Random Forest

**Consigne 7.1 — Entraînement**

In [ ]:
# VOTRE CODE ICI - Section 7.1
# from sklearn.ensemble import RandomForestClassifier
# ....
# ....
# ....

**Consigne 7.2 — Évaluation**

In [ ]:
# VOTRE CODE ICI - Section 7.2
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# print("Random Forest - Résultats:")
# print(f"Accuracy: {accuracy_score(y_test, rf_predictions):.4f}")
# print(f"Precision: {precision_score(y_test, rf_predictions):.4f}")
# print(f"Recall: {recall_score(y_test, rf_predictions):.4f}")
# print(f"F1-Score: {f1_score(y_test, rf_predictions):.4f}")
# # Importance des variables
# import pandas as pd
# feature_importance = pd.DataFrame({
#     'feature': X.columns,
#     'importance': rf_model.feature_importances_
# }).sort_values('importance', ascending=False)
# print("\nTop 10 variables importantes:")
# display(feature_importance.head(10))

## 8. Modèle 2 — Support Vector Machine (SVM)

**Consigne 8.1 — Entraînement (utiliser données normalisées)**

In [ ]:
# VOTRE CODE ICI - Section 8.1
# from sklearn.svm import SVC
# ...
# ...
# ...

**Consigne 8.2 — Évaluation**

In [ ]:
# VOTRE CODE ICI - Section 8.2
# print("SVM - Résultats:")
# print(f"Accuracy: {accuracy_score(y_test, svm_predictions):.4f}")
#....
# ...
# ...

## 9. Modèle 3 — Gradient Boosting (XGBoost)

**Consigne 9.1 — Entraînement**

In [ ]:
# VOTRE CODE ICI - Section 9.1
# try:
#     from xgboost import XGBClassifier
#     xgb_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, subsample=1.0, colsample_bytree=1.0, random_state=42, n_jobs=-1, eval_metric='logloss')
#     xgb_model.fit(X_train, y_train)
#     xgb_predictions = xgb_model.predict(X_test)
# except ImportError:
#     print("XGBoost n'est pas installé. Installez avec: pip install xgboost")
#     xgb_model = None
#     xgb_predictions = None

**Consigne 9.2 — Évaluation**

In [ ]:
# VOTRE CODE ICI - Section 9.2
# if xgb_predictions is not None:
#     print("XGBoost - Résultats:")
#     print(f"Accuracy: {accuracy_score(y_test, xgb_predictions):.4f}")
#     print(f"Precision: {precision_score(y_test, xgb_predictions):.4f}")
#     print(f"Recall: {recall_score(y_test, xgb_predictions):.4f}")
#     print(f"F1-Score: {f1_score(y_test, xgb_predictions):.4f}")

## 10. Modèle 4 — Réseau de Neurones (MLPClassifier)

**Consigne 10.1 — Entraînement (données normalisées)**

In [ ]:
# VOTRE CODE ICI - Section 10.1
# from sklearn.neural_network import MLPClassifier
# nn_model = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000, random_state=42)
# nn_model.fit(X_train_scaled, y_train)
# nn_predictions = nn_model.predict(X_test_scaled)

**Consigne 10.2 — Évaluation**

In [ ]:
# VOTRE CODE ICI - Section 10.2
# print("Réseau de Neurones - Résultats:")
# ....
# print(f"Precision: {precision_score(y_test, nn_predictions):.4f}")
# print(f"Recall: {recall_score(y_test, nn_predictions):.4f}")
# print(f"F1-Score: {f1_score(y_test, nn_predictions):.4f}")
# # Nombre d'itérations
# # ....

## 11. Comparaison des modèles

In [ ]:
# VOTRE CODE ICI - Section 11.1
# import pandas as pd
# results = []
# # Remplir en fonction des métriques calculées plus haut
# # results.append({'Model': 'Random Forest', 'Accuracy': ..., 'Precision': ..., 'Recall': ..., 'F1-Score': ...})
# # results.append({'Model': 'SVM', 'Accuracy': ..., 'Precision': ..., 'Recall': ..., 'F1-Score': ...})
# # if xgb_predictions is not None: results.append({'Model': 'XGBoost', 'Accuracy': ..., 'Precision': ..., 'Recall': ..., 'F1-Score': ...})
# # results.append({'Model': 'Neural Network', 'Accuracy': ..., 'Precision': ..., 'Recall': ..., 'F1-Score': ...})
# results_df = pd.DataFrame(results)
# display(results_df)
# # Graphique comparatif des F1-scores (optionnel)
# # import matplotlib.pyplot as plt
# # plt.figure()
# # plt.bar(results_df['Model'], results_df['F1-Score'])
# # plt.title('Comparaison des F1-scores')
# # plt.ylabel('F1-Score')
# # plt.show()

## 12. Validation croisée

In [ ]:
# VOTRE CODE ICI - Section 12.1
# from sklearn.model_selection import cross_val_score
# # Exemple avec Random Forest (remplacez par votre meilleur modèle)
# # scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1)
# # print(f"Validation croisée - F1-Score: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

## 13. Optimisation des hyperparamètres (Grid Search)

In [ ]:
# VOTRE CODE ICI - Section 13.1
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [5, 10, None],
#     'min_samples_split': [2, 5, 10]
# }
# grid_search = GridSearchCV(
#     RandomForestClassifier(random_state=42),
#     param_grid,
#     cv=3,
#     scoring='f1',
#     n_jobs=-1
# )
# grid_search.fit(X_train, y_train)
# print("Meilleurs paramètres:", grid_search.best_params_)
# print("Meilleur score CV:", grid_search.best_score_)
# best_rf = grid_search.best_estimator_
# best_rf_pred = best_rf.predict(X_test)

## 14. Analyse des erreurs

In [ ]:
# VOTRE CODE ICI - Section 14.1
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# # Exemple: matrice de confusion pour le meilleur modèle (remplacez best_rf_pred)
# # cm = confusion_matrix(y_test, best_rf_pred)
# # plt.figure(figsize=(8, 6))
# # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
# #             xticklabels=['Non-Trending', 'Trending'],
# #             yticklabels=['Non-Trending', 'Trending'])
# # plt.title('Matrice de Confusion - Meilleur Modèle')
# # plt.ylabel('Valeurs Réelles')
# # plt.xlabel('Prédictions')
# # plt.show()

In [ ]:
# VOTRE CODE ICI - Section 14.2
# # Identifier des exemples mal classifiés (adapter selon le modèle choisi)
# # errors_mask = (y_test != best_rf_pred)
# # error_indices = X_test[errors_mask].index
# # print("Exemples de vidéos mal classifiées:")
# # for idx in list(error_indices)[:5]:
# #     real_label = y_test.loc[idx]
# #     predicted_label = best_rf_pred[list(error_indices).index(idx)]  # à adapter si nécessaire
# #     print(f"Index {idx}: Réel={real_label}, Prédit={predicted_label}")

## 15. Sauvegarde & conclusions

In [ ]:
# VOTRE CODE ICI - Section 15.1
# import joblib
# # Remplacez 'best_rf' / 'scaler' par vos objets
# # joblib.dump(best_model, 'best_youtube_classifier.pkl')
# # joblib.dump(scaler, 'feature_scaler.pkl')
# # print("Modèle et scaler sauvegardés avec succès!")

### Conclusions à rédiger

1. **Meilleur modèle**  
   - Modèle: *[À compléter]*  
   - Performances: *[À compléter]*  
   - Raisons: *[À compléter]*

2. **Variables les plus importantes**  
   - *[À compléter]*

3. **Limitations**  
   - *[À compléter]*

4. **Améliorations suggérées**  
   - *[À compléter]*

5. **Apprentissages**  
   - *[À compléter]*

---
## Guide de testing pour les étudiants

**Comment tester vos implémentations :**
1. **Vérifications de base** : `.shape`, `.info()`, `.head()`  
2. **Validation des features** : pas de NaN, ratios entre 0 et 1, plages temporelles correctes  
3. **Validation des modèles** : prédictions binaires {0,1}, longueurs cohérentes, comparaisons rigoureuses  
4. **Tests de cohérence** : scores réalistes, F1 entre précision et rappel, impact de la normalisation  
5. **Debugging** : augmenter `max_iter` si nécessaire, vérifier entrées/sorties et dimensions